In [1]:
#!curl https://en.wikipedia.org/wiki/Arthur_Rimbaud

In [3]:
from pymongo import MongoClient
import pprint

import copy
import pandas as pd

import requests

from bs4 import BeautifulSoup

In [4]:
chemists_url = 'https://famouspeople.astro-seek.com/filter_occupation/?narozeni_stat=&povolani=chemists&narozeni_stat=&pohlavi=&razeni='
r = requests.get(chemists_url)

In [5]:
r.status_code

200

In [6]:
#r.content

In [7]:
client = MongoClient('localhost', 27017)
db = client.metroid
pages = db.pages

pages.insert_one({'html': r.content})

In [8]:
soup = BeautifulSoup(r.content, 'html')

In [9]:
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>ASTRO-DB FILTER  - </title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="en" http-equiv="content-language"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="index,follow" name="robots"/>
<meta content="ASTRO-DB FILTER  - " name="description"/>
<meta content="ASTRO-DB, FILTER, , , astro-seek, astroseek, horoscopes, charts, signs, zodiac, numerology, birth" name="keywords"/>
<meta content="Astro-Seek.com © 2016 - 2020" name="copyright"/>
<meta content="NdrTyPCwuaPr_26Quymem-jlfZJo4CKw8JYkkc_fsao" name="google-site-verification"/>
<meta content="Qaq5_iQ2vq2zsHrPOaZT3M-POCwZCeDJbHhKApKu-X0" name="google-site-verification"/>
<meta content="DE3A5EEEAA6EAC927A2AFC54159A80F2"

In [10]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   ASTRO-DB FILTER  -
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="en" http-equiv="content-language"/>
  <meta content="text/javascript" http-equiv="content-script-type"/>
  <meta content="text/css" http-equiv="content-style-type"/>
  <meta content="index,follow" name="robots"/>
  <meta content="ASTRO-DB FILTER  - " name="description"/>
  <meta content="ASTRO-DB, FILTER, , , astro-seek, astroseek, horoscopes, charts, signs, zodiac, numerology, birth" name="keywords"/>
  <meta content="Astro-Seek.com © 2016 - 2020" name="copyright"/>
  <meta content="NdrTyPCwuaPr_26Quymem-jlfZJo4CKw8JYkkc_fsao" name="google-site-verification"/>
  <meta content="Qaq5_iQ2vq2zsHrPOaZT3M-POCwZCeDJbHhKApKu-X0" name="google-site-verification"/>
  <meta content="DE

In [11]:
print(soup.title)

<title>ASTRO-DB FILTER  - </title>


In [12]:
div = soup.find('div', {'style': 'border-left: 1px solid #E1E1E1; border-right: 1px solid #E1E1E1; width: 758px;'})
table = div.find('table')



#The following are incorrect...

##div = soup.find('div', {'class': 'cl p5'})
##table = div.find('table')

###div = soup.find('div', {'class': 'obsah-vnitrek'})
###table = div.find('table')


In [13]:
print(table.prettify())

<table cellpadding="0" cellspacing="0" style="width: 100%; padding: 0px;">
 <tr>
  <td class="w260_p5">
   <a href="https://birthcharts.astro-seek.com/birth-chart/isaac-asimov-horoscope">
    <span>
     <strong>
      Isaac Asimov
     </strong>
    </span>
   </a>
   <span class="form-info">
    (*Jan 2, 1920)
   </span>
  </td>
  <td class="tr">
   <span class="form-info">
    writer, biochemist
   </span>
  </td>
  <td class="w30">
   <img alt="Russia" class="vlajka_flag" src="https://www.astro-seek.com/seek-images/flags/thumb/ru.gif" title="Russia"/>
  </td>
  <td class="pr2_25">
   <span class="form-info">
    RU
   </span>
  </td>
 </tr>
 <tr>
  <td colspan="4">
   <div class="cl p5">
   </div>
   <div class="result-uzivatele-prepazka" style="height: 2px; background: url(https://www.astro-seek.com/seek-images/bg_uzivatele_prepazka_sirsi_2015.gif) repeat-x 0 0px;">
   </div>
   <div class="cl p5">
   </div>
  </td>
 </tr>
 <tr>
  <td class="w260_p5">
   <a href="https://birthchar

## Webscraping Just One Page
Later I'll scrape the individual pages of each person.

In [14]:
rows = table.find_all('tr')
all_rows = []
# Think about making the following dictionary have the keys name, date, country.
empty_row = {'name_and_date': None, 'tag': None, 'flag': None, 'country': None}

# It should be... for row in rows[1::2]:... but for now, let's just do all rows.
for row in rows[0::2]:
    new_row = copy.copy(empty_row)
    columns = row.find_all('td')
    new_row['name_and_date'] = columns[0].text.strip()
    new_row['tag'] = columns[1].text.strip()
    new_row['flag'] = columns[2].text.strip()
    new_row['country'] = columns[3].text.strip()
    all_rows.append(new_row)

In [15]:
pprint.pprint(all_rows[:4])

[{'country': 'RU',
  'flag': '',
  'name_and_date': 'Isaac Asimov (*Jan 2, 1920)',
  'tag': 'writer, biochemist'},
 {'country': 'GB',
  'flag': '',
  'name_and_date': 'Michael Faraday (*Sep 22, 1791)',
  'tag': 'physicist, chemist'},
 {'country': 'FR',
  'flag': '',
  'name_and_date': 'Jean-Baptiste Dumas (*Jul 16, 1800)',
  'tag': 'chemist'},
 {'country': 'DE',
  'flag': '',
  'name_and_date': 'Friedrich Bayer (*Jun 6, 1825)',
  'tag': 'chemist, entrepreneur, founder of company Bayer AG'}]


In [16]:
db = client.metroid
chemists = db.chemists

In [17]:
for row in all_rows:
    chemists.insert_one(row)

In [18]:
rows = chemists.find()
chemists_df = pd.DataFrame(list(rows))

In [19]:
chemists_df.head(20)

,_id,country,flag,name_and_date,tag
0,5e890b6f30409fad49ad5f69,RU,,"Isaac Asimov (*Jan 2, 1920)","writer, biochemist"
1,5e890b6f30409fad49ad5f6a,GB,,"Michael Faraday (*Sep 22, 1791)","physicist, chemist"
2,5e890b6f30409fad49ad5f6b,FR,,"Jean-Baptiste Dumas (*Jul 16, 1800)",chemist
3,5e890b6f30409fad49ad5f6c,DE,,"Friedrich Bayer (*Jun 6, 1825)","chemist, entrepreneur, founder of company Baye..."
4,5e890b6f30409fad49ad5f6d,FR,,"Pierre Curie (*May 15, 1859)","physicist, chemist"
5,5e890b6f30409fad49ad5f6e,US,,"Charles Goodyear (*Dec 29, 1800)","chemist, inventor, developer of vulcanized rubber"
6,5e890b6f30409fad49ad5f6f,US,,"Dudley Robert Herschbach (*Jun 18, 1932)","chemist, Nobel Prize winner for chemistry in 1986"
7,5e890b6f30409fad49ad5f70,RU,,"Dmitri Mendeleev (*Feb 8, 1834)","Russian chemist, creator of the Periodic Table..."
8,5e890b6f30409fad49ad5f71,FR,,"Louis Pasteur (*Dec 27, 1822)","French biologist, chemist, doctor"
9,5e890b7030409fad49ad5f72,GB,,"Rupert Sheldrake (*Jun 28, 1942)","biochemist, plant physiologist, author of the ..."


In [20]:
chemists_df = chemists_df.drop('_id', axis=1)
chemists_df = chemists_df.drop('flag', axis=1)
chemists_df = chemists_df.set_index('name_and_date')
chemists_df.head()

,country,tag
name_and_date,,
"Isaac Asimov (*Jan 2, 1920)",RU,"writer, biochemist"
"Michael Faraday (*Sep 22, 1791)",GB,"physicist, chemist"
"Jean-Baptiste Dumas (*Jul 16, 1800)",FR,chemist
"Friedrich Bayer (*Jun 6, 1825)",DE,"chemist, entrepreneur, founder of company Baye..."
"Pierre Curie (*May 15, 1859)",FR,"physicist, chemist"


In [21]:
chemists_df.head(25)

,country,tag
name_and_date,,
"Isaac Asimov (*Jan 2, 1920)",RU,"writer, biochemist"
"Michael Faraday (*Sep 22, 1791)",GB,"physicist, chemist"
"Jean-Baptiste Dumas (*Jul 16, 1800)",FR,chemist
"Friedrich Bayer (*Jun 6, 1825)",DE,"chemist, entrepreneur, founder of company Baye..."
"Pierre Curie (*May 15, 1859)",FR,"physicist, chemist"
"Charles Goodyear (*Dec 29, 1800)",US,"chemist, inventor, developer of vulcanized rubber"
"Dudley Robert Herschbach (*Jun 18, 1932)",US,"chemist, Nobel Prize winner for chemistry in 1986"
"Dmitri Mendeleev (*Feb 8, 1834)",RU,"Russian chemist, creator of the Periodic Table..."
"Louis Pasteur (*Dec 27, 1822)",FR,"French biologist, chemist, doctor"


## Webscraping for each Occupation
...so that we have a 'chemists' dataframe and a 'golfers' dataframe and so on.

#### Scraping for occupations_list

In [24]:
occupations_url = 'https://famouspeople.astro-seek.com/filter_occupation/'
r = requests.get(occupations_url)

In [25]:
r.status_code

200

In [26]:
#Actually, I think I'm gonna make a new file...